# Word2Vec con Gensim

## Instalación de librerías y carga de dataset

In [ ]:
# !pip install datasets

In [1]:
from gensim.models import Word2Vec
import pandas as pd
import re
from gensim.parsing.preprocessing import strip_punctuation, strip_numeric, strip_short, stem_text
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

-------------------------

In [2]:
from datasets import load_dataset

dataset_corpus = load_dataset("large_spanish_corpus", "ParaCrawl")

README.md:   0%|          | 0.00/8.26k [00:00<?, ?B/s]

large_spanish_corpus.py:   0%|          | 0.00/3.93k [00:00<?, ?B/s]

raw.zip:   0%|          | 0.00/5.79G [00:00<?, ?B/s]

Generating train split:   0%|          | 0/15510649 [00:00<?, ? examples/s]

In [3]:
dataset_corpus

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 15510649
    })
})

In [4]:
subset = dataset_corpus['train'].select(range(1000000))

In [6]:
subset[50:60]

{'text': ['Su presencia en la tierra como poderes fantasma operando a través de sustitutos humanos serán expuestos para que todos los vean.',
  'Típica imagen de Hollywood de una invasión alienígena hostil por parte de naves aéreas extraterrestres de alta tecnología.',
  'o combinación de los anteriores?',
  'En la visión gnóstica, la divulgación o revelación no es el momento en que las Autoridades gubernamentales anuncian la presencia de extraterrestres en la tierra, haciéndolo así "oficial".',
  'Nada hecho oficial en este mundo cuenta para nada más que para otra táctica en el engaño globalista, avanzando en la agenda de las Autoridades.',
  'Sin embargo, para que los arcontes hagan su movimiento final, deberán revelarse.',
  'Debido a que su naturaleza es el engaño, lo harán de forma engañosa.',
  'Incluso al mostrar su presencia, jugando su carta de triunfo sobre la especie humana, ellos van a engañar, porque eso es todo lo que ellos hacen, todo lo que pueden hacer.',
  'La prueba 

In [8]:
from paths import EXTERNAL_DIR

In [9]:
import nltk
nltk.download('stopwords', download_dir=EXTERNAL_DIR)
nltk.download('punkt', download_dir=EXTERNAL_DIR)

from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jssdev/Dev/Learning/Platzi/platzi-
[nltk_data]     embeddings/data/external...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/jssdev/Dev/Learning/Platzi/platzi-
[nltk_data]     embeddings/data/external...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## Procesamiento de texto

In [ ]:

    # extrae el texto de la entrada



        # Convierte el texto a minúsculas


        # Elimina URLs


        # Elimina las menciones @ y '#' de las redes sociales


        # Elimina los caracteres de puntuación


        # Elimina los números


        # Elimina las palabras cortas


        # Elimina las palabras comunes (stop words)




    # Devuelve el texto limpio



## Carga y uso de modelo de embeddings Word2Vec

In [ ]:
model = Word2Vec()

# Podemos guardar el modelo para uso futuro
model.save("word2vec.model")

In [ ]:
##comida, ser, reina, television


## Almacenamiento de embeddings